# December 12, 2021

https://adventofcode.com/2021/day/12

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque

In [2]:
def format_data( data ):
    lines = data.split("\n")

    links = defaultdict(set)

    for line in lines:
        p1,p2 = line.split("-")
        links[p1].add(p2)
        links[p2].add(p1)

    return links

def deep_copy( data ):
    return [ [x for x in line] for line in data ]

In [3]:
with open("../data/2021/12.txt", "r") as f:
    data_str = f.read()
data = format_data( data_str )

In [4]:
test1_str = '''start-A
start-b
A-c
A-b
b-d
A-end
b-end'''

test1 = format_data(test1_str)
test1

defaultdict(set,
            {'start': {'A', 'b'},
             'A': {'b', 'c', 'end', 'start'},
             'b': {'A', 'd', 'end', 'start'},
             'c': {'A'},
             'd': {'b'},
             'end': {'A', 'b'}})

In [5]:
test2_str = '''dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc'''

test2 = format_data(test2_str)
test2

defaultdict(set,
            {'dc': {'HN', 'LN', 'end', 'kj', 'start'},
             'end': {'HN', 'dc'},
             'HN': {'dc', 'end', 'kj', 'start'},
             'start': {'HN', 'dc', 'kj'},
             'kj': {'HN', 'dc', 'sa', 'start'},
             'LN': {'dc'},
             'sa': {'kj'}})

In [6]:
test3_str = '''fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW'''

test3 = format_data(test3_str)
test3

defaultdict(set,
            {'fs': {'DX', 'end', 'he', 'pj'},
             'end': {'fs', 'zg'},
             'he': {'DX', 'RW', 'WI', 'fs', 'pj', 'zg'},
             'DX': {'fs', 'he', 'pj', 'start'},
             'start': {'DX', 'RW', 'pj'},
             'pj': {'DX', 'RW', 'fs', 'he', 'start', 'zg'},
             'zg': {'RW', 'end', 'he', 'pj', 'sl'},
             'sl': {'zg'},
             'RW': {'he', 'pj', 'start', 'zg'},
             'WI': {'he'}})

# Part 1

In [7]:
def must_be_unique( cavern ):
    return not cavern.isupper()

def is_end( cavern ):
    return cavern == "end"

def find_paths( links ):
    paths = deque([["start"]])
    solutions = []
    while len(paths) > 0:
        path = paths.pop()
        for next_step in links[path[-1]]:
            if is_end(next_step):
                solutions.append( path + [next_step] )
            elif not must_be_unique(next_step) or next_step not in path:
                paths.appendleft( path + [next_step] )

    return solutions

In [8]:
find_paths(test1)

[['start', 'b', 'end'],
 ['start', 'A', 'end'],
 ['start', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'end'],
 ['start', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'end']]

In [9]:
sol = find_paths(test2)
print(len(sol))
sol

19


[['start', 'HN', 'end'],
 ['start', 'dc', 'end'],
 ['start', 'HN', 'dc', 'end'],
 ['start', 'kj', 'HN', 'end'],
 ['start', 'kj', 'dc', 'end'],
 ['start', 'dc', 'HN', 'end'],
 ['start', 'HN', 'kj', 'HN', 'end'],
 ['start', 'HN', 'kj', 'dc', 'end'],
 ['start', 'HN', 'dc', 'HN', 'end'],
 ['start', 'kj', 'HN', 'dc', 'end'],
 ['start', 'kj', 'dc', 'HN', 'end'],
 ['start', 'dc', 'kj', 'HN', 'end'],
 ['start', 'HN', 'kj', 'HN', 'dc', 'end'],
 ['start', 'HN', 'kj', 'dc', 'HN', 'end'],
 ['start', 'HN', 'dc', 'kj', 'HN', 'end'],
 ['start', 'kj', 'HN', 'dc', 'HN', 'end'],
 ['start', 'dc', 'HN', 'kj', 'HN', 'end'],
 ['start', 'HN', 'kj', 'HN', 'dc', 'HN', 'end'],
 ['start', 'HN', 'dc', 'HN', 'kj', 'HN', 'end']]

In [10]:
sol = find_paths(test3)
print(len(sol))

226


In [11]:
sol = find_paths(data)
len(sol)

5756

# Part 2

In [12]:
def is_small( cavern ):
    return not cavern.isupper()

class Path:
    # path = list of caverns
    # flag = True if list of caverns includes two smalls
    def __init__( self, path, next_step=[], flag=False ):
        self.path = path + next_step
        self.flag = flag

    def __repr__( self ):
        f = str(self.flag*1)
        return "("+f+") ["+", ".join([str(x) for x in self.path]) + "]"
    
    def __str__( self ):
        return __repr__( self )
    def __getitem__(self, i):
        return self.path[i]
    
    def try_to_visit( self, cavern ):
        # can't go back home home!
        if cavern == "start":
            return None
        
        small_flag = is_small(cavern)
        # If it's not small, just append to path
        if not small_flag:
            return Path( self.path, [cavern], self.flag )
        
        if cavern in self.path:
            # we visited this cavern. Pnly go backif flag is False
            # in this case set flag True because we will then have a double-small
            if not self.flag:
                return Path( self.path, [cavern], True )
            else:
                return None
        else:
            # new cavern. safe to append. Flag doesn't change
            return Path( self.path, [cavern], self.flag )
        
def find_paths2( links ):
    paths = deque( [Path(["start"])] )
    solutions = []

    while len(paths) > 0:
        path = paths.pop()
        for next_step in links[path[-1]]:
            new_path = path.try_to_visit( next_step )
            if new_path is None:
                continue
            if next_step == "end":
                solutions.append( new_path )
            else:
                paths.appendleft( new_path )

    return solutions        

In [13]:
sol = find_paths2(test1)
print(len(sol))
sol[:10]

36


[(0) [start, b, end],
 (0) [start, A, end],
 (0) [start, b, A, end],
 (0) [start, A, b, end],
 (1) [start, b, A, b, end],
 (1) [start, b, d, b, end],
 (0) [start, A, c, A, end],
 (0) [start, A, b, A, end],
 (0) [start, b, A, c, A, end],
 (1) [start, b, A, b, A, end]]

In [14]:
sol = find_paths2(test2)
print(len(sol))

103


In [15]:
sol = find_paths2(test3)
print(len(sol))

3509


In [16]:
sol = find_paths2(data)
print(len(sol))

144603
